#  Dependecies

In [298]:
!pip3 install -U ncps pytorch-lightning ucimlrepo

In [299]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import LTC, CfC
import pytorch_lightning as pl
import torch
import torch.utils.data as data
from torchvision.datasets import MNIST
import torchvision.transforms as transforms



# Ozone dataset

**Ozone Day Prediction**

 The objective of task is to forecast ozone days, i.e., days when the local ozone concentration exceeds
 a critical level. Input features consist of wind, weather, and solar radiation readings.


 The original dataset ”Ozone Level Detection Data Set” was taken from the UCI repository (Dua and Graff 2017) consists
 of daily data points collected by the Texas Commission on Environmental Quality (TCEQ). We split the 6-years period into
 overlapping sequences of 32 days. A day was labeled as ozone day if, for at least 8 hours, the exposure to ozone exceeded
 80 parts per billion. Inputs consist of 73 features, including wind, temperature, and solar radiation data. The binary predictor
 variable has a prior of 6.31%, i.e., expresses a 1:15 imbalance.

 For the training procedure, we weighted the **cross-entropy loss**
 at each day, depending on the label.

 Labels representing an ozone day were assigned 15 times the weight of a non-ozone day.
 Moreover, we reported the F1-score instead of standard accuracy (higher score is better).

 In roughly 27% of all samples, some of the input features were missing. To not disrupt the continuity of the collected data,
 we set all missing features to zero.

 Note that such zeroing of some input features potentially negatively affects the performance
 of our RNNmodels compared to non-recurrent approaches and filtering out the missing data. Consequently, ensemble methods
 and model-based approaches, i.e., methods that leverage domain knowledge (Zhang and Fan 2008), can outperform the end
to-end RNNs studied in our experiment.

We randomly split all sub-sequences into training (75%), validation (10%), and test
 (15%) set.

In [300]:
# Download ozone dataset
from ucimlrepo import fetch_ucirepo

# fetch dataset
ozone_level_detection = fetch_ucirepo(id=172)
ozone_level_detection.data.keys()


dict_keys(['ids', 'features', 'targets', 'original', 'headers'])

In [301]:
# data (as pandas dataframes)
ids_df = ozone_level_detection.data.ids
X_df = ozone_level_detection.data.features
y_df = ozone_level_detection.data.targets

In [302]:
ids_df

,Dataset,Date
0,8hr,1/1/1998
1,8hr,1/2/1998
2,8hr,1/3/1998
3,8hr,1/4/1998
4,8hr,1/5/1998
...,...,...
5065,1hr,12/27/2004
5066,1hr,12/28/2004
5067,1hr,12/29/2004
5068,1hr,12/30/2004


In [303]:
mask_8hr = ids_df["Dataset"] == "8hr"

In [304]:
ids_df = ids_df[mask_8hr]
X_df = X_df[mask_8hr]
y_df = y_df[mask_8hr]

In [305]:
print(X_df.isnull().sum().sum()) # check the overall number of missing values

14937


In [306]:
X_df.fillna(0, inplace=True)

<ipython-input-306-60869a6b1cb0>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [307]:
print(X_df.isnull().sum().sum()) # check the overall number of missing values


0


In [308]:
ids_df

,Dataset,Date
0,8hr,1/1/1998
1,8hr,1/2/1998
2,8hr,1/3/1998
3,8hr,1/4/1998
4,8hr,1/5/1998
...,...,...
2529,8hr,12/27/2004
2530,8hr,12/28/2004
2531,8hr,12/29/2004
2532,8hr,12/30/2004


In [309]:
X_df

,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,WSR9,...,T50,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp
0,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,-15.5,0.15,10.67,-1.56,5795.0,-12.10,17.90,10330.0,-55.0,0.00
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,-14.5,0.48,8.39,3.84,5805.0,14.05,29.00,10275.0,-55.0,0.00
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,-15.9,0.60,6.94,9.80,5790.0,17.90,41.30,10235.0,-40.0,0.00
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,-16.8,0.49,8.73,10.54,5775.0,31.15,51.70,10195.0,-40.0,2.08
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,0.3,0.4,0.5,0.5,0.2,0.3,0.4,0.4,1.3,2.2,...,-12.4,0.07,7.93,-4.41,5800.0,-25.60,21.80,10295.0,65.0,0.00
2530,1.0,1.4,1.1,1.7,1.5,1.7,1.8,1.5,2.1,2.4,...,-12.0,0.04,5.95,-1.14,5845.0,-19.40,19.10,10310.0,15.0,0.00
2531,0.8,0.8,1.2,0.9,0.4,0.6,0.8,1.1,1.5,1.5,...,-11.8,0.06,7.80,-0.64,5845.0,-9.60,35.20,10275.0,-35.0,0.00
2532,1.3,0.9,1.5,1.2,1.6,1.8,1.1,1.0,1.9,2.0,...,-10.8,0.25,7.72,-0.89,5845.0,-19.60,34.20,10245.0,-30.0,0.05


In [310]:
y_df

,Class
0,0
1,0
2,0
3,0
4,0
...,...
2529,0
2530,0
2531,0
2532,0


In [311]:
X_np = X_df.to_numpy()
y_np = y_df.to_numpy()

print(f"shape of X: {X_np.shape}")
print(f"shape of y: {y_np.shape}")


shape of X: (2534, 72)
shape of y: (2534, 1)


In the old implementation they standardize using all data, not train set

In [312]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_np = scaler.fit_transform(X_np)

In [313]:
def cut_in_sequences(x,y,seq_len,inc=1):

    sequences_x = []
    sequences_y = []

    for s in range(0,x.shape[0] - seq_len,inc):
        start = s
        end = start+seq_len
        sequences_x.append(x[start:end])
        sequences_y.append(y[start:end])

    return np.stack(sequences_x,axis=0), np.stack(sequences_y,axis=0)

X_sequences, y_sequences = cut_in_sequences(X_np,y_np,seq_len=32,inc=16)
print(f"shape of X_sequences: {X_sequences.shape}")
print(f"shape of y_sequences: {y_sequences.shape}")


shape of X_sequences: (157, 32, 72)
shape of y_sequences: (157, 32, 1)


In [314]:
total_seqs = X_sequences.shape[0]
print("Total number of training sequences: {}".format(total_seqs))

Total number of training sequences: 157


In [315]:
permutation = np.random.RandomState(23489).permutation(total_seqs)

In [316]:
valid_size = int(0.1*total_seqs)
test_size = int(0.15*total_seqs)
train_size = total_seqs - valid_size - test_size
print(f"Train sequences:{train_size}, validation sequences: {valid_size}, test sequences: {test_size}")

Train sequences:119, validation sequences: 15, test sequences: 23


In [317]:
# sequences for train, validation and test in numpy
X_val = X_sequences[permutation[:valid_size],:]
y_val = y_sequences[permutation[:valid_size],:]
X_test = X_sequences[permutation[valid_size:valid_size+test_size],:]
y_test = y_sequences[permutation[valid_size:valid_size+test_size],:]
X_train = X_sequences[permutation[valid_size+test_size:],:]
y_train = y_sequences[permutation[valid_size+test_size:],:]

In [318]:
# convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test =  torch.tensor(X_test , dtype=torch.float32)
y_test =  torch.tensor(y_test , dtype=torch.float32)
X_val =   torch.tensor(X_val  , dtype=torch.float32)
y_val =   torch.tensor(y_val  , dtype=torch.float32)

In [319]:
# move to gpu if possible
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("device selected: cuda")
else:
    device = torch.device('cpu')
    print("device selected: cpu")

X_train = X_train.to(device)
y_train = y_train.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)

X_val = X_val.to(device)
y_val = y_val.to(device)


device selected: cuda


In [320]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False) # Not to shuffle for reproducibility
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle validation
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # No need to shuffle test


In [321]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class SequenceLearner(pl.LightningModule):
    def __init__(self, model, lr=0.001, pos_weight=15.0):
        """
        :param model: The PyTorch model to train.
        :param lr: Learning rate.
        :param pos_weight: Weight multiplier for positive class (class 1).
        """
        super().__init__()
        self.model = model
        self.lr = lr
        self.pos_weight = torch.tensor(pos_weight)

        # Use BCEWithLogitsLoss for numerical stability
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)

        # Initialize empty lists to store predictions and labels during testing
        self.probabilities_validation = []
        self.labels_validation = []
        self.probabilities_test = []
        self.labels_test = []

    def forward(self, x):
        return self.model(x)

    def compute_loss(self, logits, y):
        """ Compute weighted binary cross-entropy loss. """
        return self.loss_fn(logits.view(-1), y.view(-1))

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits, _ = self.model.forward(x)
        loss = self.compute_loss(logits, y)

        # Log the loss
        self.log("train_loss", loss, prog_bar=True)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits, _ = self.model.forward(x)
        loss = self.compute_loss(logits, y)

        # Store predictions and labels for metrics
        self.probabilities_validation.extend(list(torch.sigmoid(logits).cpu().detach().numpy().reshape(-1)))
        self.labels_validation.extend(list(y.cpu().detach().numpy().reshape(-1)))

        # Log the loss
        self.log("val_loss", loss, prog_bar=True)
        return {"loss": loss}

    def on_validation_epoch_end(self):
        """ Calculate metrics at the end of validation epoch. """
        probs = np.array(self.probabilities_validation)
        labels = np.array(self.labels_validation)
        preds = (probs > 0.5).astype(int)

        acc = accuracy_score(labels, preds)
        precision = precision_score(labels, preds)
        recall = recall_score(labels, preds)
        f1 = f1_score(labels, preds)

        # Log metrics
        self.log("val_accuracy", acc, prog_bar=True)
        self.log("val_precision", precision, prog_bar=True)
        self.log("val_recall", recall, prog_bar=True)
        self.log("val_f1", f1, prog_bar=True)

        # Clear lists for next epoch
        self.probabilities_validation.clear()
        self.labels_validation.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits, _ = self.model.forward(x)
        loss = self.compute_loss(logits, y)
        # Store predictions and labels
        self.probabilities_test.append(torch.sigmoid(logits).cpu().detach())
        self.labels_test.append(y.cpu())
        return {"loss": loss}


    def configure_optimizers(self):
        return optim.Adam(self.model.parameters(), lr=self.lr)


In [322]:
# Adjust LTC model parameters
in_features = 72  # Number of features in the dataset
out_features = 1  # Output for BCELoss (logit)
RNN_state_size = 32  # Number of units in the RNN layer


Here we can finally create a LTCCell and make use of the predefined sparse wiring structures of the keras-ncp package. For simplicity we will just define a fully-connected RNN

In [333]:
wiring = AutoNCP(units=RNN_state_size, output_size=out_features, sparsity_level=0.5)
ltc_model = LTC(input_size=in_features, units=wiring).to(device)
cfc_model = CfC(input_size=in_features, units=wiring).to(device)

model = SequenceLearner(ltc_model, lr=0.001).to(device)
trainer = pl.Trainer(
    logger=pl.loggers.CSVLogger("log"),
    max_epochs=200,
    gradient_clip_val=1,  # Clip gradient to stabilize training
)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [334]:
trainer.fit(model, train_loader, val_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | LTC               | 16.9 K | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
13.6 K    Trainable params
3.3 K     Non-trainable params
16.9 K    Total params
0.068     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


In [335]:
model.probabilities_test = []
model.labels_test = []
trainer.test(model, test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]

In [336]:
probabilities = []
for i_batch in range(len(model.probabilities_test)):
    for i_sequence in range(len(model.probabilities_test[i_batch])):
        probabilities.extend(list(map(float, model.probabilities_test[i_batch][i_sequence].reshape(-1))))
probabilities = np.array(probabilities)

In [337]:
labels = []
for i_batch in range(len(model.labels_test)):
    for i_sequence in range(len(model.labels_test[i_batch])):
        labels.extend(list(map(float, model.labels_test[i_batch][i_sequence].reshape(-1))))
labels = np.array(labels)

In [338]:
predictions = np.zeros_like(labels)
predictions[probabilities>0.5] = 1

In [339]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=labels, mode='lines', name='Ground Truth'))
fig.add_trace(go.Scatter(y=predictions, mode='lines', name='Predictions'))
fig.add_trace(go.Scatter(y=probabilities, mode='lines', name='Probablities'))
fig.update_layout(title="Ozone Day Prediction", template="simple_white")
fig.update_yaxes(title="Ozone Day", dtick=0.5, showgrid=True)
fig.show()

In [ ]:
# prompt: I have classification. I have numpy array of predicted classes and numpy array of actual classes. I need F1 score, precision, recall, accuracy

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Assuming 'predictions' and 'labels' are your numpy arrays
f1 = f1_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
accuracy = accuracy_score(labels, predictions)

print(f"{"F1 Score":>10}: {round(f1,3)}")
print(f"{"Precision":>10}: {round(precision,3)}")
print(f"{"Recall":>10}: {round(recall,3)}")
print(f"{"Accuracy":>10}: {round(accuracy,3)}")


F1 Score: 0.533
Precision: 0.374
Recall: 0.93
Accuracy: 0.905


In [341]:
trainer.logged_metrics

{}

In [1]:
import pandas as pd
import plotly.graph_objects as go

metrics_ltc = pd.read_csv("logs/ozone_ltc.csv")
metrics_cfc = pd.read_csv("logs/ozone_cfc.csv")
df_val_metrics_ltc = metrics_ltc.drop("train_loss", axis=1).dropna()
df_val_metrics_cfc = metrics_cfc.drop("train_loss", axis=1).dropna()


df_val_metrics_cfc

,epoch,step,val_accuracy,val_f1,val_loss,val_precision,val_recall
0,0,3,0.689583,0.300469,1.196466,0.184971,0.800
1,1,7,0.781250,0.378698,1.114913,0.248062,0.800
2,2,11,0.820833,0.448718,1.033256,0.301724,0.875
3,3,15,0.845833,0.486111,0.982575,0.336538,0.875
4,4,19,0.872917,0.527132,0.956120,0.382022,0.850
...,...,...,...,...,...,...,...
210,195,783,0.972917,0.826667,0.957715,0.885714,0.775
211,196,787,0.972917,0.826667,0.957786,0.885714,0.775
212,197,791,0.972917,0.826667,0.957855,0.885714,0.775
213,198,795,0.972917,0.826667,0.957924,0.885714,0.775


In [2]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_val_metrics_ltc["epoch"], y=df_val_metrics_ltc["val_f1"], mode='lines', name='LTC'))
fig.add_trace(go.Scatter(x=df_val_metrics_cfc["epoch"], y=df_val_metrics_cfc["val_f1"], mode='lines', name='CfC'))
fig.add_trace(go.Scatter(x=[0,200], y=[0.302, 0.302], mode='lines', name='Best result of the article', line=dict(dash="dash")))
fig.update_layout(title="Ozone Level Detection: validation F1-score", template="simple_white")
fig.update_yaxes(title="F1-score", showgrid=True, range = [0,1] , dtick=0.1)
fig.update_xaxes(title="Epoch", showgrid=True, range=[0,201], dtick=20)
fig.update_layout(width=576, height=384)
# Move legend inside the plot
fig.update_layout(
    legend=dict(
        x=0.5,   # Horizontal position (closer to 1 moves it to the right)
        y=0.0,   # Vertical position (closer to 1 moves it to the top)
        bgcolor="rgba(255, 255, 255, 0.5)"  # Optional: semi-transparent background
    )
)
fig.show()

In [3]:
fig.write_image("Ozone_f1.pdf", format="pdf")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [11]:
! pip install -U kaleido

     --------------------------------------- 65.9/65.9 MB 10.6 MB/s eta 0:00:00
